## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Imports

In [13]:
import numpy as np
import cv2
import glob
import matplotlib
#matplotlib.use('qt5agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


%matplotlib qt5
#%matplotlib qt

## Utility functions

In [14]:
def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def calibrate(images):
    """Computes the camera calibration using chessboard images"""
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.


    # Step through the list and search for chessboard corners
    for fname in images:
        print("processing image: {}".format(fname))
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[0:2], None, None)
    return cameraMatrix, distCoeffs


def undistort_image(img, mtx, dist):
    """Apply a distortion correction to raw image"""
    import matplotlib.image as mpimg
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def hls_transform(img):
    """Applies the hsl transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

def sobel_operator(img, dir='x'):
    gray = grayscale(img)
    if (dir == 'x'):
        return cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        return cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    
def sobel_scale(img_sobel):
    """Absolute the derivative to accentuate lines away from horizontal/vertical??"""
    abs_sobel = np.absolute(img_sobel) 
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    return scaled_sobel
    
def select_yellow(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower = np.array([20,60,60])
    upper = np.array([38,174, 250])
    mask = cv2.inRange(hsv, lower, upper)
    return mask

def select_white(img):
    lower = np.array([202,202,202])
    upper = np.array([255,255,255])
    mask = cv2.inRange(img, lower, upper)
    return mask

def apply_mask(img, mask):
    result = cv2.bitwise_and(img,img,mask=mask)
    return result                    

def threshold_image(img, thresh=(20, 100)):
    s_binary = np.zeros_like(img)
    s_binary[(img >= thresh[0]) & (img <= thresh[1])] = 1
    return s_binary

def binary_image_transform_yellow_white_lane_lines(img):
    yellow_mask = select_yellow(img)
    yellow_image = apply_mask(img, yellow_mask)
    ret,yellow_binary = cv2.threshold(grayscale(yellow_image),127,255,cv2.THRESH_BINARY)

    white_mask = select_white(img)
    white_image = apply_mask(img, white_mask)
    ret,white_binary = cv2.threshold(grayscale(white_image),127,255,cv2.THRESH_BINARY)

    combined_binary = np.zeros_like(white_binary)
    combined_binary[(yellow_binary > 0) | (white_binary > 0)] = 1
#     plot_2_images(img, combined_binary)
    return combined_binary

def plot_2_images(img1, img2, title1='original',title2='processed'):
    import matplotlib.pyplot as plt

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(48, 18))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2, cmap='gray')
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    
def plot_image_title(img, title='', text=''):
    
    font = {'family': 'serif',
            'color':  'white',
            'weight': 'normal',
            'size': 28,
            }
        
    f, ax = plt.subplots(1, 1, figsize=(48, 18))
    f.tight_layout()
    ax.imshow(img)
    plt.text(2, 75.65, text, fontdict=font)
    ax.set_title(title, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#     plt.savefig(output_file)

def binary_image_transform(img):
    """Uses gradients to create a binary image"""

    # Threshold x gradient
    sobel_x = sobel_operator(img, dir='x')
    s_x_binary = threshold_image(sobel_scale(sobel_x), thresh=(20, 100))

    # Threshold color channel
    hls = hls_transform(img)
    s_channel = hls[:,:,2]
    s_binary = threshold_image(s_channel, thresh=(170, 255))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(s_x_binary)
    combined_binary[(s_binary == 1) | (s_x_binary == 1)] = 1
    return combined_binary

def unwarp(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M

def hist(img):
    return np.sum(img[img.shape[0]//2:,:], axis=0)


def polyfit(img, y, x, order=2):
    """Fit polynomial"""
    fit = np.poly1d(np.polyfit(y, x, order))

    #y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    fit_x = fit(ploty)
    return fit_x


def detect_lane_lines(binary_warped, debug = False,nwindows = 9, margin = 100, minpix = 50):
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)

    histogram = hist(binary_warped)

    out_img = None
    if (debug==True):
        # For debugging, an output image to visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint


    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):

        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        if (debug==True):
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    if (len(rightx)==0 or len(leftx)==0):
        return [], [], [], []
        
#     print("len(rightx) {}".format(len(rightx)))
#     print("len(leftx) {}".format(len(leftx)))
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    left_fitx = polyfit(binary_warped, lefty, leftx, 2)
    right_fitx = polyfit(binary_warped, righty, rightx, 2)

    if (debug==True):
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
    return left_fit, right_fit, left_fitx, right_fitx

def detect_lane_lines_subsequent_images(binary_warped, left_fit, right_fit, debug=False, margin = 100):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    #TODO improve this 
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fitx = polyfit(binary_warped, lefty, leftx, 2)
    right_fitx = polyfit(binary_warped, righty, rightx, 2)

    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    if (debug==True):
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
    return left_fit, right_fit, left_fitx, right_fitx

        
def curvature(binary_warped, left_fitx, right_fitx):    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    y_eval = np.max(ploty)

    leftx = left_fitx
    rightx = right_fitx

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    # Now our radius of curvature is in meters
    center_of_lanes = ((right_fitx[-1] - left_fitx[-1]) //2 + left_fitx[-1]) * xm_per_pix
    center_of_car = (binary_warped.shape[1] // 2) * xm_per_pix
    return left_curverad, right_curverad, (center_of_lanes - center_of_car)

def warp_detected_lines_onto_original(original, warped, left_fitx, right_fitx, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original.shape[1], original.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
#     plt.imshow(result)
    return result

## Compute the camera calibration using chessboard images

In [ ]:
images = glob.glob('../camera_cal/calibration*.jpg')
cameraMatrix, distCoeffs = calibrate(images)

## Apply a distortion correction to a raw images

In [ ]:
img = mpimg.imread('../test_images/test5.jpg')
undist = undistort_image(img, cameraMatrix, distCoeffs)
plot_2_images(img, undist)

## Create a thresholded binary image.

In [ ]:
# binary_image = binary_image_transform(undist)
# plot_2_images(img, binary_image)

In [ ]:
binary_image = binary_image_transform_yellow_white_lane_lines(undist)
plot_2_images(undist, binary_image)

## Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
img_size = (binary_image.shape[1], binary_image.shape[0])
src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 40, img_size[1]],
    [(img_size[0] / 2 + 65), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])
warped, M = unwarp(binary_image, src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

print(src)
# plot_2_images(undist, warped)

warped_undist, M2 = unwarp(undist, src, dst)

src2 = np.int32(src.reshape((-1,1,2)))
dst2 = np.int32(dst.reshape((-1,1,2)))

cv2.polylines(undist,[src2],True,(255,0,0))
cv2.polylines(warped_undist,[dst2],True,(255,0,0))

plot_2_images(undist, warped_undist)

## Detect lane pixels and fit to find the lane boundary.

In [ ]:
left_fit, right_fit, left_fitx, right_fitx = detect_lane_lines(warped, debug=True)
left_fit, right_fit, left_fitx, right_fitx = detect_lane_lines_subsequent_images(warped, left_fit, right_fit, debug=True)

## Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
left_curverad, right_curverad, diff_center = curvature(warped, left_fitx, right_fitx)
print(left_curverad, 'm', right_curverad, 'm', diff_center)

## Warp the detected lane boundaries back onto the original image.

In [ ]:
result = warp_detected_lines_onto_original(undist, warped, left_fitx, right_fitx, Minv)
plot_2_images(img, result)

## Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# title = "Radius of curvature = {:f}(m)\nVehicle is {:f}m {!s} of center ".format(np.mean(left_curverad,right_curverad), np.abs(diff_center), 'left')
text = 'Radius of curvature = {:f}(m)\nVehicle is {:f}m {!s} of center'.format(np.mean([left_curverad,right_curverad]), np.abs(diff_center), 'left' if diff_center > 0 else 'right')
plot_image_title(result, '', text)

## Define a class to receive the characteristics of each line detection

In [ ]:
class Line():
    def __init__(self):
        #loss count
        self.loss_count = 0
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients of the last n fits of the line
        self.recent_fit = []
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    #update the stats
    def update(self, fit, fitx, radius_of_curvature, line_base_pos, warped):
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        n = len(self.recent_xfitted)
        
        if (n > 10):
            removed_fitx = self.recent_xfitted.pop(0)
            removed_fit = self.recent_fit.pop(0)
        
        self.recent_xfitted.append(fitx)
        self.bestx = np.mean(self.recent_xfitted, axis=0,keepdims=True)
        self.recent_fit.append(fit)
        self.best_fit = np.mean(self.recent_fit)
        self.radius_of_curvature = radius_of_curvature
        self.line_base_pos = line_base_pos 
        self.diffs = self.current_fit - fit
        self.current_fit = fit
        self.allx = fitx
        self.ally = ploty
        self.detected = True
        self.loss_count = np.max([self.loss_count-1, 0])
    def reset():
        self.detected = False  
        self.recent_xfitted = [] 
        self.bestx = None     
        self.best_fit = None  
        self.current_fit = [np.array([False])]  
        self.radius_of_curvature = None 
        self.line_base_pos = None 
        self.diffs = np.array([0,0,0], dtype='float') 
        self.allx = None  
        self.ally = None
        self.loss_count=0

## Pipeline

In [ ]:
def sanity_check(leftLine, rightLine, left_fit, right_fit, left_fitx, right_fitx, left_curverad, right_curverad, diff_center):
    
    #Checking that lines have similar curvature
    if ( (left_curverad-right_curverad) / right_curverad > 2):
        print("left_curverad, right_curverad")
        print(left_curverad, right_curverad)
        return False

    #Checking that lines are separated by approximately the right distance horizontally
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    if ( ((right_fitx[-1]-left_fitx[-1]) * xm_per_pix - 3.7) > 5e-2):
        print("right_fitx[-1], left_fitx[-1")
        print(right_fitx[-1], left_fitx[-1])
        return False

#     #Checking that lines are roughly parallel
    upper = right_fitx[0] - left_fitx[0]
    lower = right_fitx[-1] - left_fitx[-1]
    if ( (upper-lower) / lower > 11e-2):
        print("upper, lower")
        print(upper, lower)
        return False
#     print('great, passed sanity checks!')
    
    return True

In [ ]:
def sanity_check_update_lines(left_fit, right_fit, left_fitx, right_fitx, warped, leftLine, rightLine):
    if (len(left_fit) == 0):
        leftLine.detected = False
        leftLine.loss_count += 1

        rightLine.detected = False
        rightLine.loss_count += 1

        return 0., 0., 0.
    left_curverad, right_curverad, diff_center = curvature(warped, left_fitx, right_fitx)

    if (sanity_check(leftLine, rightLine, left_fit, right_fit, left_fitx, right_fitx, left_curverad, right_curverad, diff_center)):
        leftLine.update(left_fit, left_fitx, left_curverad, diff_center, warped)
        rightLine.update(right_fit, right_fitx, right_curverad, diff_center, warped)
    else:
        leftLine.detected = False
        leftLine.loss_count += 1

        rightLine.detected = False
        rightLine.loss_count += 1
        
    return left_curverad, right_curverad, diff_center

In [ ]:
def pipeline(img, simple=True):
    """
    1) Sanity Check
    2) Look-Ahead Filter
    3) Reset
    4) Smoothing
    5) Drawing
    """
    undist = undistort_image(img, cameraMatrix, distCoeffs)
    binary_image = binary_image_transform_yellow_white_lane_lines(undist)
    warped, M = unwarp(binary_image, src, dst)
    
    if (simple):
        left_fit, right_fit, left_fitx, right_fitx = detect_lane_lines(warped)
        left_curverad, right_curverad, diff_center = sanity_check_update_lines(left_fit, right_fit, left_fitx, right_fitx, warped, leftLine, rightLine)
    else:    
        if (len(leftLine.recent_xfitted) > 0 & leftLine.loss_count < 2 & len(rightLine.recent_xfitted) > 0  & rightLine.loss_count < 5):
            left_fit, right_fit, left_fitx, right_fitx = detect_lane_lines_subsequent_images(warped, leftLine.best_fit, rightLine.best_fit)
            left_curverad, right_curverad, diff_center = sanity_check_update_lines(left_fit, right_fit, left_fitx, right_fitx, warped, leftLine, rightLine)        
        else:
            print('resetting....')
            leftLine.reset
            rightLine.reset
            left_fit, right_fit, left_fitx, right_fitx = detect_lane_lines(warped)
            left_curverad, right_curverad, diff_center = sanity_check_update_lines(left_fit, right_fit, left_fitx, right_fitx, warped, leftLine, rightLine)
        
    if (left_curverad == 0. or right_curverad == 0.):
        return img
    result = warp_detected_lines_onto_original(undist, warped, leftLine.bestx, rightLine.bestx, Minv)
    text1 = 'Radius of curvature = {:f}(m)'.format(np.mean([left_curverad,right_curverad]))
    text2 = 'Vehicle is {:f}m {!s} of center'.format(np.abs(diff_center), 'left' if diff_center > 0 else 'right')
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result,text1,(10,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,text2,(10,95), font, 1,(255,255,255),2,cv2.LINE_AA)
    
    return result

In [ ]:
import os
test_image_file_names = os.listdir("../test_images/")
leftLine = Line()
rightLine = Line()
for image_file_name in test_image_file_names:
    print(image_file_name)
    image = mpimg.imread("../test_images/" + image_file_name)    
    result = pipeline(image, simple=True)
    mpimg.imsave("../output_images/" + image_file_name, result)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    result = pipeline(image)
    return result

output = '../output_images/project_video.mp4'
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    result = pipeline(image)
    return result

output = '../output_images/challenge_video.mp4'
clip1 = VideoFileClip("../challenge_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    result = pipeline(image)
    return result

output = '../output_images/harder_challenge_video.mp4'
clip1 = VideoFileClip("../harder_challenge_video.mp4")
clip1.set_end(5.23)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

## The goals / steps of this project are the following:

##Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier

##Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.

##Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.

##Implement a sliding-window technique and use your trained classifier to search for vehicles in images.

##Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.

##Estimate a bounding box for vehicles detected.

In [1]:
import matplotlib.image as mpimg
import numpy as np
import cv2
from skimage.feature import hog
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# from lesson_functions import *
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split


/Users/ajaffer/anaconda/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

In [5]:
car_images = glob.glob('../data/vehicles/**/*.png')
non_car_images = glob.glob('../data/non-vehicles/**/*.png')
cars = []
notcars = []
for image in car_images:
    cars.append(image)
for image in non_car_images:
    notcars.append(image)

print("read {} car images".format(len(cars)))
print("read {} notcars images".format(len(notcars)))


read 8792 car images
read 8968 notcars images


In [21]:
color_space = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
# orient = 9  # HOG orientations
orient = 12  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
# hog_channel = 0 # Can be 0, 1, 2, or "ALL"
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
# hist_bins = 256    # Number of histogram bins
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
# y_start_stop = [None, None] # Min and max in y to search in slide_window()
y_start_stop = [400, None] # Min and max in y to search in slide_window()

In [7]:
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

/Users/ajaffer/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [19]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
model = svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()


# Save to a pickle file.
import pickle
pickle.dump( model, open( "model.p", "wb" ) )



Using: 12 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 7872
5.15 Seconds to train SVC...
Test Accuracy of SVC =  0.9899


In [22]:
import pickle

model = pickle.load( open( "model.p", "rb" ) )



image = mpimg.imread('../test_images/test1.jpg')
draw_image = np.copy(image)

# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
# image = image.astype(np.float32)/255

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(128, 128), xy_overlap=(0.5, 0.5))

hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

# plt.imshow(window_img)
plot_2_images(window_img,window_img)



/Users/ajaffer/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
